# Load DeepFly3D Data

In [1]:
%load_ext autoreload
%autoreload 2
from load import *
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
from liftpose.vision_3d import transform_frame
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# decleare data parameters
par_train = {'data_dir' : '/data/LiftPose3D/fly_tether/data_DF3D', # change the path 
       'out_dir'        : '/data/LiftPose3D/fly_tether/',
       'train_subjects' : [1],
       'test_subjects'  : [6,7],
       'actions'        : ['all'],
       'cam_id'         : [2,5]}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

# Load 2D data
train_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

# Load 3D data
train_3d, _, rcams_train = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_3d, _, rcams_test = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)


train_3d = transform_frame(train_3d, rcams_train)
test_3d = transform_frame(test_3d, rcams_test)

# Train LiftPose3D Network on DeepFly3D Data

In [2]:
from liftpose.main import train as lp3d_train

lp3d_train(train_2d=train_2d, test_2d=test_2d, train_3d=train_3d, test_3d=test_3d, 
           **{k:par[k] for k in ['roots', 'target_sets', 'out_dir']})

[main.py:180]:INFO:Saving pre-processed 2D data at /home/gosztolai/Dropbox/github/LiftPose3D/examples/universal_lifter/out/stat_2d.pth.tar.
[main.py:202]:INFO:Saving pre-processed 3D data at /home/gosztolai/Dropbox/github/LiftPose3D/examples/universal_lifter/outstat_3d.pth.tar.
[main.py:229]:INFO:Starting training model.
[main.py:238]:DEBUG:
==================Options=================
[main.py:239]:DEBUG:{   'batch_size': 128,
    'data_dir': '/home/gosztolai/Dropbox/github/LiftPose3D/examples/universal_lifter/out',
    'dropout': 0.5,
    'epochs': 30,
    'exp': '',
    'is_train': True,
    'job': 8,
    'linear_size': 1024,
    'load': None,
    'lr': 0.001,
    'lr_decay': 5000,
    'lr_gamma': 0.9,
    'max_norm': True,
    'noise': None,
    'num_stage': 2,
    'out': '/home/gosztolai/Dropbox/github/LiftPose3D/examples/universal_lifter/out',
    'out_dir': '/home/gosztolai/Dropbox/github/LiftPose3D/examples/universal_lifter/out',
    'predict': False,
    'procrustes': False,
   

Epoch 000 | LR  0.00100 | Loss Test  0.00000 | Loss Train  0.43083|: 100%|██████████| 1646/1646 [00:09<00:00, 169.61it/s]
Epoch 001 | LR  0.00100 | Loss Test  0.11283 | Loss Train  0.16664|: 100%|██████████| 1646/1646 [00:09<00:00, 170.88it/s]
Epoch 002 | LR  0.00100 | Loss Test  0.09578 | Loss Train  0.15179|: 100%|██████████| 1646/1646 [00:09<00:00, 171.92it/s]
Epoch 003 | LR  0.00090 | Loss Test  0.08857 | Loss Train  0.14180|: 100%|██████████| 1646/1646 [00:09<00:00, 171.99it/s]
Epoch 004 | LR  0.00090 | Loss Test  0.07957 | Loss Train  0.13657|: 100%|██████████| 1646/1646 [00:09<00:00, 175.80it/s]
Epoch 005 | LR  0.00090 | Loss Test  0.07550 | Loss Train  0.13213|: 100%|██████████| 1646/1646 [00:09<00:00, 170.49it/s]
Epoch 006 | LR  0.00081 | Loss Test  0.07268 | Loss Train  0.12796|: 100%|██████████| 1646/1646 [00:09<00:00, 173.84it/s]
Epoch 007 | LR  0.00081 | Loss Test  0.07132 | Loss Train  0.12560|: 100%|██████████| 1646/1646 [00:09<00:00, 171.74it/s]
Epoch 008 | LR  0.00081 

In [ ]:
%matplotlib inline
from liftpose.plot import read_log_train, plot_log_train
epoch, lr, loss_train, loss_test, err_test = read_log_train(par['out_dir'])
plot_log_train(plt.gca(), loss_train, loss_test, epoch)

# Run Trained LiftPose3D Network on the Test Data

In [ ]:
from liftpose.main import test as lp3d_test
lp3d_test(par['out_dir'])

# Visualize Predictions

In [ ]:
from liftpose.postprocess import load_test_results
data = torch.load(os.path.join(par['out_dir'], "test_results.pth.tar"))
stat_2d, stat_3d = (
    torch.load(os.path.join(par['out_dir'], "stat_2d.pth.tar")),
    torch.load(os.path.join(par['out_dir'], "stat_3d.pth.tar")),
)
test_3d_gt, test_3d_pred = load_test_results(data, stat_2d, stat_3d)

In [ ]:
# https://stackoverflow.com/a/38865534/7554774
# conda install ipympl
#%matplotlib widget
%matplotlib inline
from liftpose.plot import plot_pose_3d

fig = plt.figure(figsize=plt.figaspect(1), dpi=100)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=200, azim=60)

t = 900
plot_pose_3d(ax=ax, tar=test_3d_gt[t], 
            pred=test_3d_pred[t], 
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"])